In [138]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl
import glob

today = date.today()
# today = today - datetime.timedelta(days=2)
today = today.strftime("%m.%d.20%y")
today

'02.01.2022'

In [139]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('FanDuel-NBA*.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [236]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [237]:
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position
0,71094-55062,C,Nikola,Nikola Jokic,Jokic,57.222728,44.0,11300,DEN@MIN,DEN,MIN,GTD,Toe,NaN,NaN,NaN,C
1,71094-40199,PF/SF,Giannis,Giannis Antetokounmpo,Antetokounmpo,54.754761,42.0,11000,WAS@MIL,MIL,WAS,NaN,NaN,NaN,NaN,NaN,SF/PF
2,71094-9644,SG/SF,James,James Harden,Harden,49.078573,42.0,10500,BKN@PHO,BKN,PHO,GTD,Hand,NaN,NaN,NaN,SG/SF
3,71094-67708,PG/SG,Dejounte,Dejounte Murray,Murray,47.922222,45.0,10200,GS@SA,SA,GS,NaN,NaN,NaN,NaN,NaN,PG/SG
4,71094-58462,C,Karl-Anthony,Karl-Anthony Towns,Towns,44.958138,43.0,10000,DEN@MIN,MIN,DEN,NaN,NaN,NaN,NaN,NaN,C


In [238]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [239]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id
0,Jordan Poole,GSW,SAS,35.0,PG/SG,"5,300",38.9,12.7,47.9,80.6,MAIN,Jordan Poole,PG/SG,71094-110322
1,Juan Toscano-Anderson,GSW,SAS,29.0,SF/PF,"3,500",29.5,10.5,38.6,71.8,MAIN,Juan Toscano-Anderson,SF/PF,71094-117779
2,Nikola Jokic,DEN,MIN,34.0,C,"11,300",58.3,8.1,67.7,29.0,MAIN,Nikola Jokic,C,71094-55062
3,Devin Booker,PHX,BKN,38.0,SG,"9,000",46.9,5.9,56.9,19.8,MAIN,Devin Booker,SG,71094-58460
4,Moses Moody,GSW,SAS,35.0,SG,"3,500",24.6,5.6,32.3,16.0,MAIN,Moses Moody,SG,71094-157802


In [240]:
# ## Unnests Roster Positions into each Row (explode in Python) ##


# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Position'].astype(str)
df['Roster Position'] = df['Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Roster Position
0,Jordan Poole,GSW,SAS,35.0,PG/SG,5300.0,38.9,12.7,47.9,80.6,MAIN,Jordan Poole,PG/SG,71094-110322,"[PG, SG]"
1,Juan Toscano-Anderson,GSW,SAS,29.0,SF/PF,3500.0,29.5,10.5,38.6,71.8,MAIN,Juan Toscano-Anderson,SF/PF,71094-117779,"[SF, PF]"
2,Nikola Jokic,DEN,MIN,34.0,C,11300.0,58.3,8.1,67.7,29.0,MAIN,Nikola Jokic,C,71094-55062,[C]
3,Devin Booker,PHX,BKN,38.0,SG,9000.0,46.9,5.9,56.9,19.8,MAIN,Devin Booker,SG,71094-58460,[SG]
4,Moses Moody,GSW,SAS,35.0,SG,3500.0,24.6,5.6,32.3,16.0,MAIN,Moses Moody,SG,71094-157802,[SG]


In [241]:
df.to_excel("FD_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
# os.system("FD_final_PlayerPool_ETR_"+str(today)+".xlsx")

# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_C


In [242]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [243]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [244]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

In [245]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['FD Position'].iloc[i]) for i in range(len(df))])
    
prob += get_position_sum(player_vars, df, 'PG') <= 3
prob += get_position_sum(player_vars, df, 'PG') >= 2
prob += get_position_sum(player_vars, df, 'SG') <= 3
prob += get_position_sum(player_vars, df, 'SG') >= 2
# prob += get_position_sum(player_vars, df, 'G') <= 2
# prob += get_position_sum(player_vars, df, 'G') >= 2
# prob += get_position_sum(player_vars, df, 'SF') <= 2
prob += get_position_sum(player_vars, df, 'SF') >= 2
# prob += get_position_sum(player_vars, df, 'PF') <= 2
prob += get_position_sum(player_vars, df, 'PF') >= 2
# prob += get_position_sum(player_vars, df, 'F') <= 2
# prob += get_position_sum(player_vars, df, 'F') >= 2
prob += get_position_sum(player_vars, df, 'C') <= 1
prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 2
# prob += get_position_sum(player_vars, df, 'UTIL') <= 2



In [246]:
# total salary constraint

prob += pulp.lpSum(df['FD Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 60000

In [247]:
# finally, specify the goal
prob += pulp.lpSum([df['FD Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [248]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [249]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['FD Position'], row.Player, row.Team, row['FD Salary'], row['FD Points'])
        total_salary_used += row['FD Salary']
        mean_AvgPointsPerGame += row['FD Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

PG/SG Jordan Poole GSW 5300.0 38.9
SF/PF Juan Toscano-Anderson GSW 3500.0 29.5
C Nikola Jokic DEN 11300.0 58.3
SG Moses Moody GSW 3500.0 24.6
SF Jonathan Kuminga GSW 4400.0 27.7
PF/SF Giannis Antetokounmpo MIL 11000.0 53.7
PG Spencer Dinwiddie WAS 6300.0 34.6
PG/SG Jrue Holiday MIL 7500.0 37.6
SF/PF Kyle Kuzma WAS 7200.0 36.4


(60000.0, 341.3)